In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install import-ipynb
import import_ipynb
!pip install colorama

Mounted at /content/drive
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=021c0b8aeaf12fe44fdf98573942ea6c72f329b22be9687986da7867378aef03
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [2]:
%cd drive
%cd 'My Drive'
%cd 'RL'
%cd 'Homework'
%cd 'Homework 4'


/content/drive
/content/drive/My Drive
/content/drive/My Drive/RL
/content/drive/My Drive/RL/Homework
/content/drive/My Drive/RL/Homework/Homework 4


In [15]:
'''Needed Libraries and Environment'''
import Environment
import Modified_Environment
import matplotlib.pyplot as plt
import numpy as np
import math
import random
from collections import defaultdict
import sys
import itertools
import matplotlib
import pandas as pd
import pickle

'''Constants Defined'''
STATE = 0
ACTION = 1
REWARD = 2
MAX_T = 100000
TOTAL_EPISODES = 2000
gamma = 0.9
MIN = 0.1
DECAY_FACTOR = 0.99

In [40]:
class OnPolicyMCAgent():
  
    def __init__(self, env, gamma):
      self.env = env
      self.Q = defaultdict(lambda: np.zeros(env.action_space.n))
      self.actions = env.action_space.n
      self.gamma = gamma
      self.returns_sum = defaultdict(float)
      self.returns_count = defaultdict(float)
      self.episode_information = []
      self.MC_episode_scores = []


    def epsilon_greedy_policy(self, state, epsilon):
        epsilon_policy = np.ones(self.actions, dtype=float) * epsilon / self.actions
        best_action = np.argmax(self.Q[state])
        epsilon_policy[best_action] += (1.0 - epsilon)
        return epsilon_policy


    def reach_island(self):
        #epsilon = 0.5 #<-- uncomment if you want to use decaying epsilon 
        #epsilon = 0.3 #<-- uncomment if you want to use constant epsilon
        for episode in range(1, TOTAL_EPISODES + 1):
            episode_score = 0
            if episode % 2 == 0:
                print("\rEpisode {}/{}.".format(episode, TOTAL_EPISODES), end="")
                sys.stdout.flush()
                #epsilon = max(epsilon * DECAY_FACTOR, MIN)#<-- uncomment if you want to use decaying epsilon
            state = self.env.reset()
            self.episode_information = []
            done = False
            for t in itertools.count():
                probs = self.epsilon_greedy_policy(state, epsilon)
                action = np.random.choice(np.arange(len(probs)), p = probs)
                state_prime, reward, done, _ = self.env.step(action)
                episode_score += reward
                self.episode_information.append((state, action, reward))
                if done or t > MAX_T :break
                state = state_prime
            self.on_policy_returns()
            self.MC_episode_scores.append(episode_score)

    def on_policy_returns(self):
        unique_state_action_pairs_in_episode = set([(x[STATE], x[ACTION]) for x in self.episode_information])
        for state, action in unique_state_action_pairs_in_episode:
            first_occurence_idx = next(idx for idx, x in enumerate(self.episode_information) if x[STATE] == state and x[ACTION] == action)
            G = sum([x[REWARD]*(self.gamma**i) for i, x in enumerate(self.episode_information[first_occurence_idx:])])
            self.returns_sum[(state, action)] += G
            self.returns_count[(state, action)] += 1.0
            self.Q[state][action] = self.returns_sum[(state, action)] / self.returns_count[(state, action)]

In [41]:
# env =  Environment.GridworldEnv() #<--uncomment for normal environment
#env =  Modified_Environment.GridworldEnv() # <--uncomment for modified environment (bonus part)
agent = OnPolicyMCAgent(env, gamma)
agent.reach_island()

Episode 2000/2000.

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(range(TOTAL_EPISODES), agent.MC_episode_scores, color = '#633974', label = '$\epsilon$ = from $0.5$ to $0.1$')
plt.xlabel('episodes ->')
plt.ylabel('epsiode score ->')
plt.title('On-policy Monte Carlo')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
window_size = 50
smoothed_score = pd.Series(agent.MC_episode_scores).rolling(window_size , min_periods = window_size).mean()
plt.plot(smoothed_score, color = '#633974', label = '$\epsilon$ = from $0.5$ to $0.1$')
#plt.annotate(smoothed_score.iloc[-1], xy=(2000, -14), xytext=(2000, -14), color = '#633974',) # <--uncomment for modified environment (bonus part)
plt.xlabel("epsiode ->")
plt.ylabel("epsiode score (smoothed) -> ")
plt.title('On-policy Monte Carlo')
plt.legend()
plt.show()

In [44]:
#i wrote my reward data into a .txt file to use it later.

# with open('MC_bonus_scores', 'wb') as fp:
#     pickle.dump(agent.MC_episode_scores, fp)